## SAT-NAE

### Aim
Given a SAT problem, solve it such that for each clause, have at least one variable to be set to false.

### Example

We aim to solve a 3-SAT problem of the form

$(x_1 \lor x_2 \lor \neg x_3) \land (x_3 \lor \neg x_1 \lor \neg x_6) \land (\neg x_2 \lor x_4 \lor x_5) \land (\neg x_4 \lor \neg x_5 \lor x_6)$

such that each clause $C$ must satisfy that for the three variables in $C$, at least one is true, and at least one is false. By setting $w_i = (-1)^{x_i}$, we aim to solve:

$\displaystyle \max\limits_{w_i \in \{-1,1\}} \sum\limits_{c\in C}\frac{3}{4}-\frac{1}{4}(w_iw_j + w_iw_k + w_jw_k) \equiv \frac{3}{4}|C| - \frac{1}{4} \min\limits_{w_i \in \{-1, 1\}} \sum\limits_{c\in C}(w_iw_j + w_iw_k + w_jw_k)$.

Thus, our problem becomes:

$\displaystyle \min\limits_{w_i \in \{-1, 1\}} \sum\limits_{c\in C}(w_iw_j + w_iw_k + w_jw_k) + M\sum\limits_{i=1}^n (w_i w_{i+n})$

where the second sum accounts for a penalty where we want to ensure that $w_i$ and $\neg w_i$ indeed hold different values (and M is a sufficiently large positive number)

## Try for QAOA Form
$\sum\limits_{c \in C} (Z_iZ_j+Z_iZ_k+Z_jZ_k) + M\sum\limits_{i=1}^n(Z_iZ_{i+n})$

## Encode SAT Formula

- Number of variables = $n$
- Number of qubits that we will use: $2n$
- Encode the variables into their indices (i.e. $x_1 \implies 1$, $x_5 \implies 5$)
- Replace the negation of variable $i$ with ($i+n$)

### Example
- Input: (1 or 2 or not 3) and (not 1 or 2 or 3)
- Problem properties: $n = 3 \implies 2n = nqubits = 6$ 
- Output: $\left[[1,2,6], [4,2,3]\right]$

In [1]:
## Some constants
NEG_TEXT = "not "
AND_TEXT = " and "
OR_TEXT = " or "
M_FACTOR = 10 # arbitrary, but has to be large enough

## Parser Code

In [2]:
"""
Parses the input formula (string) and outputs a list of clauses, where each clause contains a variable (negated if necessary)
Clauses must be of the form (x_1 or not x_2 or x_3) divided by the " and " keyword. x_1, x_2 and x_3 have to be indices
(i.e. x_1 === 0, x_2 === 1, x_3 === 3)
"""
def parse_formula(formula, num_variables):
    clauses_text = formula.split(AND_TEXT)
    # split the clauses by the OR_TEXT keyword to parse them individually
    clauses_simplified = list(map(lambda clause_text: clause_text.split(OR_TEXT), clauses_text))
    for i in range(len(clauses_simplified)):
        for j in range(len(clauses_simplified[i])):
            # Remove the parentheses
            clauses_simplified[i][j] = clauses_simplified[i][j].replace('(', '')
            clauses_simplified[i][j] = clauses_simplified[i][j].replace(')', '')
            if clauses_simplified[i][j].startswith(NEG_TEXT):
                var_index = clauses_simplified[i][j]
                negated = clauses_simplified[i][j].startswith(NEG_TEXT)
                if negated:
                    # assign the index i+n to the negated variable i
                    clauses_simplified[i][j] = int(clauses_simplified[i][j].split(" ")[-1])+num_variables
        clauses_simplified[i] = list(map(lambda x: int(x), clauses_simplified[i]))

    return clauses_simplified

### Example

We aim to solve a 3-SAT problem of the form

$(x_1 \lor x_2 \lor \neg x_3) \land (x_3 \lor \neg x_1 \lor \neg x_6) \land (\neg x_2 \lor x_4 \lor x_5) \land (\neg x_4 \lor \neg x_5 \lor x_6)$

such that each clause $C$ must satisfy that for the three variables in $C$, at least one is true, and at least one is false. By setting $w_i = (-1)^{x_i}$, we aim to solve:

### General Case Formula

$\displaystyle \max\limits_{w_i \in \{-1,1\}} \sum\limits_{c\in C}\frac{3}{4}-\frac{1}{4}(w_iw_j + w_iw_k + w_jw_k) \equiv \frac{3}{4}|C| - \frac{1}{4} \min\limits_{w_i \in \{-1, 1\}} \sum\limits_{c\in C}(w_iw_j + w_iw_k + w_jw_k)$.

Thus, our problem becomes:

$\displaystyle \min\limits_{w_i \in \{-1, 1\}} \sum\limits_{c\in C}(w_iw_j + w_iw_k + w_jw_k) + M\sum\limits_{i=1}^n (w_i w_{i+n})$

where the second sum accounts for a penalty where we want to ensure that $w_i$ and $\neg w_i$ indeed hold different values (and M is a sufficiently large positive number).

In this notebook, I chose `M = M_FACTOR*num_variables`

## Try for QAOA Form
$\sum\limits_{c \in C} (Z_iZ_j+Z_iZ_k+Z_jZ_k) + M\sum\limits_{i=1}^n(Z_iZ_{i+n})$

## Create the Problem Hamiltonian

In [3]:
from qiskit.opflow import I, X, Y, Z

"""
Prepares a zero Hamiltonian of for num_variables qubits (i.e. a 2^n x 2^n zero matrix)
"""
def create_zero_hamiltonian(num_variables):
    zero_H = I-I # 2x2 0-matrix
    result = zero_H
    for i in range(2*num_variables-1):
        result = result^zero_H
    return result

"""
Creates the 3SAT-NAE problem Hamiltonian (to later be used by QAOA)
"""
def create_problem_hamiltonian(num_variables, problem_input):
    h_p_p1 = create_zero_hamiltonian(num_variables)
    # Create first sum (triangle max-cut Hamiltonian per clause)
    for clause in problem_input:
        clause_H = create_zero_hamiltonian(num_variables)
        pairs = [(clause[0], clause[1]), (clause[0], clause[2]), (clause[1], clause[2])]
        for pair in pairs:
            pairH = 1
            for i in range(2*num_variables):
                if pair[0] == i or pair[1] == i:
                    pairH = pairH^Z
                else:
                    pairH = pairH^I
            clause_H += pairH
        h_p_p1 += clause_H

    # Create second sum (penalty term)
    h_p_p2 = create_zero_hamiltonian(num_variables)
    for i in range(num_variables):
        temp_h_p_p2 = 1
        for j in range(2*num_variables):
            if j == i or j == num_variables+i:
                temp_h_p_p2 = temp_h_p_p2^Z
            else:
                temp_h_p_p2 = temp_h_p_p2^I
        h_p_p2 += temp_h_p_p2
    h_p_p2 *= M_FACTOR*num_variables
    return h_p_p1 + h_p_p2

## Verify the consistency and problem constraint satisfactions

In [5]:
"""
Verifies if the variable assignments are consistent (i.e. make sure that x and not(x) evaluate to different values)
Returns a boolean (True if the assignments are consistent, False otherwise)
"""
def verify_consistency(assignments, num_variables, show_info=True):
    for i in range(num_variables):
        # Check if the assignment to x and indeed not x are the same
        if assignments[i] == assignments[i+num_variables]:
            if show_info:
                print(f"Inconsistent at index {i}")
            return False
    if show_info:
        print("Consistent!")
    return True

In [6]:
"""
Verifies if the formula is satsified or not (with additional checks for the NAE constraint).
Returns a boolean (True if both constraints are satisfied, False otherwise)
"""
def verify(result, sat_nae, num_variables, show_info=True):
    clause_values = []
    nae_holds = True
    for clause_index in range(len(sat_nae)):
        clause_value = False
        clause_and = True # do an AND inside each clause to check for NAE
        for i in range(3): # Because 3 variables per clause
            if sat_nae[clause_index][i] < num_variables:
                result[sat_nae[clause_index][i]]
                clause_value = clause_value or bool(result[sat_nae[clause_index][i]])
                clause_and = clause_and and bool(result[sat_nae[clause_index][i]])
            else:
                clause_value = clause_value or not bool(result[sat_nae[clause_index][i]-num_variables])
                clause_and = clause_and and bool(result[sat_nae[clause_index][i]])
        clause_values.append(clause_value)
        if clause_and:
            if show_info:
                print("Found NAE breach in clause " + str(clause_index))
            nae_holds = False
    final_value = True
    nae_value = True
    for i in range(len(clause_values)):
        final_value = final_value and clause_values[i]
    if final_value and nae_holds:
        if show_info:
            print("SAT-NAE!")
        return True
    else:
        if show_info:
            print("Not SAT-NAE! :(")
            if final_value and not nae_holds:
                print("Does not satisfy the NAE constraint.")
            elif not final_value and nae_value:
                print("Does not satisfy the formula but NAE holds.")
            else:
                print("Not SAT and not NAE")
        return False

In [37]:
# Cell copied from https://qiskit.org/documentation/tutorials/algorithms/05_qaoa.html
from collections import OrderedDict
from qiskit.opflow import StateFn
import numpy as np


def sample_most_likely(state_vector):
    """Compute the most likely binary string from state vector.
    Args:
        state_vector (numpy.ndarray or dict): state vector or counts.
    Returns:
        numpy.ndarray: binary string as numpy.ndarray of ints.
    """
    if isinstance(state_vector, (OrderedDict, dict)):
        # get the binary string with the largest count
        binary_string = sorted(state_vector.items(), key=lambda kv: kv[1])[-1][0]
        x = np.asarray([int(y) for y in reversed(list(binary_string))])
        return x
    elif isinstance(state_vector, StateFn):
        binary_string = list(state_vector.sample().keys())[0]
        x = np.asarray([int(y) for y in reversed(list(binary_string))])
        return x
    else:
        n = int(np.log2(state_vector.shape[0]))
        k = np.argmax(np.abs(state_vector))
        x = np.zeros(n)
        for i in range(n):
            x[i] = k % 2
            k >>= 1
        return x

## User input for the SAT-NAE formula

In [8]:
num_variables = int(input("Enter the number of variables in the formula: "))
formula = input("Enter a SAT formula (example: (0 or 1 or not 2) and (not 0 or 3 or 4): ")
# example: (0 or 1 or not 2) and (2 or not 0 or not 5) and (not 1 or 3 or 4) and (not 3 or not 4 or 5)
sat_nae = parse_formula(formula, num_variables)

In [9]:
h_p = create_problem_hamiltonian(num_variables, sat_nae)
print(h_p)

0.0 * IIIIIIIIIIII
+ 0.0 * IIIIIIIIIIII
+ 1.0 * ZZIIIIIIIIII
+ 1.0 * ZIIIIIIIZIII
+ 1.0 * IZIIIIIIZIII
+ 0.0 * IIIIIIIIIIII
+ 1.0 * IIZIIIZIIIII
+ 1.0 * IIZIIIIIIIIZ
+ 1.0 * IIIIIIZIIIIZ
+ 0.0 * IIIIIIIIIIII
+ 1.0 * IIIZIIIZIIII
+ 1.0 * IIIIZIIZIIII
+ 1.0 * IIIZZIIIIIII
+ 0.0 * IIIIIIIIIIII
+ 1.0 * IIIIIIIIIZZI
+ 1.0 * IIIIIZIIIZII
+ 1.0 * IIIIIZIIIIZI
+ 0.0 * IIIIIIIIIIII
+ 60.0 * ZIIIIIZIIIII
+ 60.0 * IZIIIIIZIIII
+ 60.0 * IIZIIIIIZIII
+ 60.0 * IIIZIIIIIZII
+ 60.0 * IIIIZIIIIIZI
+ 60.0 * IIIIIZIIIIIZ


## Run QAOA

In [40]:
from qiskit import IBMQ
from qiskit_optimization.runtime import QAOAClient
from qiskit.algorithms.optimizers import COBYLA
from qiskit.providers.fake_provider import FakeGuadalupeV2
from qiskit.algorithms import QAOA
from qiskit.utils import QuantumInstance

IBMQ.load_account()


provider = IBMQ.get_provider(hub="ibm-q-utokyo", group="internal", project="hirashi-jst")
backend = provider.get_backend("ibmq_qasm_simulator")

# Running using Qiskit Runtime
qaoa = QAOAClient(optimizer=COBYLA(), provider=provider, backend=backend)

# instance = QuantumInstance(backend=FakeGuadalupeV2(), shots=8192)
#qaoa = QAOA(optimizer=COBYLA(), quantum_instance=instance, reps=10)

ibmqfactory.load_account:WARNING:2023-04-24 17:21:32,512: Credentials are already in use. The existing account in the session will be replaced.


In [41]:
result = qaoa.compute_minimum_eigenvalue(h_p)

## Output the results

In [42]:
full_output = list(sample_most_likely(result.eigenstate))[::-1]
reduced_output = full_output[:num_variables]
print(f"full output: {full_output}")
print(f"result: {reduced_output} (input was \"{formula}\")")

full output: [0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1]
result: [0, 0, 0, 0, 1, 0] (input was "(0 or 1 or not 2) and (2 or not 0 or not 5) and (not 1 or 3 or 4) and (not 3 or not 4 or 5)")


## Verify if the gotten result is feasible or not

In [43]:
verify_consistency(full_output, num_variables) and verify(full_output, sat_nae, num_variables)

Consistent!
SAT-NAE!


True